# 01: Exploring the API | RTEM Hackathon

## Import Packages
Let's start by importing the stuff we need.
This includes packages to work with JSON data, as well as the Onboard Client

In [1]:
# Import the packages we need to do our analysis
import json

import pandas as pd
from onboard.client import RtemClient

/Users/danlsn/Documents/Projects/Hackathons/rtem-hackathon/venv/lib/python3.9/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


## Be Safe With Your Creds
Just because this is a Hackathon and a personal project doesn't mean we get to flippantly broadcast our API Keys to
the world. I want to publish this code to GitHub so let's be responsible and import our keys using configparser.

In [2]:
# Load API Key using configparser
import configparser

config = configparser.ConfigParser()
config.read('../config.ini')
api_key = config['DEFAULT']['API_KEY']

## Use Cached Data?
Some of these client requests take a long time to return data, so in the interests of time I saved local versions by
writing DataFrames to JSON files.

In [3]:
# If use_local set to true, try to find the local file and use that.
use_local = True

## Initialise the Onboard Client
Then check to see if it's working properly.

In [4]:
# Initialise Onboard Client
client = RtemClient(api_key=api_key)

# Check that everything is in order
whoami = client.whoami()
print(f"""
    Result: {whoami['result'].upper()}
    Scope: {whoami['apiKeyScopes']}
    Version: {whoami['apiVersion']}
    User: {whoami['userInfo']['full_name']}
    Org: {whoami['userInfo']['org_short_name']}
""")


    Result: OK
    Scope: ['buildings:read', 'general', 'auth']
    Version: 2022-04-14
    User: Daniel Lawson
    Org: RTEM Submission



## Get Information About the Buildings


In [5]:
# Create a DataFrame with all buildings
all_buildings = pd.json_normalize(client.get_all_buildings()).set_index('id')
all_buildings

,org_id,name,address,sq_ft,image_src,bms_manufacturer,bms_product_name,bms_version,timezone,status,...,info.sunend,info.geoCity,info.geoState,info.m2fstart,info.satstart,info.sunstart,info.yearBuilt,info.geoCountry,info.weatherRef,info.customerType
id,,,,,,,,,,,,,,,,,,,,,
140,5,83483,None,21869.0,None,None,None,None,America/New_York,LIVE,...,,New York,NY,,,,,US,,Commercial Office
191,5,94414,None,198057.0,None,None,None,None,America/New_York,LIVE,...,23:00,Victor,NY,07:30,07:30,07:30,,US,,Commercial Retail
231,5,89496,None,NaN,None,None,None,None,America/New_York,LIVE,...,,New York,NY,,,,,US,,
248,5,116742,None,127000.0,None,None,None,None,America/New_York,LIVE,...,,New York,NY,,,,,US,,Multifamily
249,5,90366,None,120000.0,None,None,None,None,America/New_York,LIVE,...,,New York,NY,,,,,US,,Commercial Retail
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,5,94413,None,198057.0,None,None,None,None,America/New_York,LIVE,...,23:00,Henrietta,NY,07:30,07:30,07:30,,US,,Commercial Retail
370,5,95457,None,421316.0,None,None,None,None,America/New_York,LIVE,...,,New York,NY,,,,,US,,Hospitality
383,5,79528,None,75000.0,None,None,None,None,America/New_York,LIVE,...,NaN,Purchase,New York,04:00:00,NaN,NaN,1971,US,NaN,Commercial Office


### What Kind of Buildings Are There?
There are several different types of buildings that participate in the RTEM Program.

In [6]:
# Get Building Types to DataFrame
building_types = all_buildings[['info.customerType']].rename(columns={'info.customerType': 'building_type'})
# Copy DataFrame to the Clipboard for README.md
pd.io.clipboards.to_clipboard(building_types.value_counts().to_markdown(), excel=False)
building_types.value_counts()

building_type     
Multifamily           69
Commercial Retail     50
Commercial Office     46
Healthcare            16
Food/Beverage         13
K-12 School            9
College/University     7
Hospitality            7
Not For Profit         5
                       2
Chemicals              1
Manufacturing          1
dtype: int64

In [7]:
all_buildings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 229 entries, 140 to 399
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   org_id             229 non-null    int64  
 1   name               229 non-null    object 
 2   address            0 non-null      object 
 3   sq_ft              202 non-null    float64
 4   image_src          0 non-null      object 
 5   bms_manufacturer   0 non-null      object 
 6   bms_product_name   0 non-null      object 
 7   bms_version        0 non-null      object 
 8   timezone           229 non-null    object 
 9   status             229 non-null    object 
 10  equip_count        229 non-null    int64  
 11  point_count        229 non-null    int64  
 12  info.floors        224 non-null    object 
 13  info.m2fend        224 non-null    object 
 14  info.satend        223 non-null    object 
 15  info.sunend        223 non-null    object 
 16  info.geoCity       226 n

In [8]:
all_units = pd.json_normalize(client.get_all_units()).set_index('id')
all_units

,name_long,name_abbr,data_type,raw_encoding,display_encoding,qudt,unit_type,raw_encoding.0,raw_encoding.1,display_encoding.0,display_encoding.1,raw_encoding.2,display_encoding.2
id,,,,,,,,,,,,,
23,Unknown,?,None,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN
21,Enable,Enable,Binary,NaN,NaN,None,None,[disable],[enable],Disable,Enable,NaN,NaN
22,Heat/Cool/Off,H/C/O,Enum,NaN,NaN,None,None,[off],[heat],Off,Heat,[cool],Off
31,Ordinal Value,Ordinal,Ordinal,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN
34,Ampere,A,Continuous,NaN,NaN,http://qudt.org/vocab/unit/A,http://qudt.org/vocab/quantitykind/ElectricCur...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,Standard Cubic Foot,SFt3,Continuous,NaN,NaN,http://qudt.org/vocab/unit/SFT3-NAT-GAS,http://qudt.org/vocab/quantitykind/Energy,NaN,NaN,NaN,NaN,NaN,NaN
86,Kilojoules per Kilogram,kJ/kg,Continuous,NaN,NaN,http://qudt.org/vocab/unit/KiloJ-PER-KiloGM,http://qudt.org/vocab/quantitykind/SpecificEnergy,NaN,NaN,NaN,NaN,NaN,NaN
87,Pulse,pulse,Continuous,NaN,NaN,http://qudt.org/vocab/unit/NUM,http://qudt.org/vocab/quantitykind/Dimensionless,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
equip_types = pd.json_normalize(client.get_equipment_types())
equip_types[['id', 'name_long', 'name_abbr']].sort_values('name_long').set_index('id')

,name_long,name_abbr
id,,
49,Air Dryer,DRYER
12,Air Handling Unit,AHU
45,Battery,BATT
19,Boiler,BLR
20,Chilled Water Plant,CHWS
21,Chiller,CH
48,Cogeneration Plant,COGEN
47,Computer Room Air Conditioner,CRAC
22,Condenser,COND


In [10]:
# Beware: this request takes a WHILE
if use_local:
    with open('../api/equipment/all.json', 'r') as f:
        j = json.load(f)
        all_equipment = pd.json_normalize(j)
else:
    all_equipment = pd.json_normalize(client.get_all_equipment())
all_equipment.set_index('id')

,building_id,equip_id,suffix,equip_type_name,equip_type_id,equip_type_abbr,equip_type_tag,equip_subtype_name,equip_subtype_id,equip_subtype_tag,flow_order,floor_num_physical,floor_num_served,area_served_desc,equip_dis,parent_equip,child_equip,points,tags
id,,,,,,,,,,,,,,,,,,,
28797,441,boiler-1,1,Boiler,19,BLR,boiler,None,NaN,None,1,None,None,None,None,[28803],[],"[{'id': 310029, 'building_id': 441, 'last_upda...","[boiler, hvac]"
28798,441,boiler-2,2,Boiler,19,BLR,boiler,None,NaN,None,1,None,None,None,None,[28803],[],"[{'id': 310079, 'building_id': 441, 'last_upda...","[boiler, hvac]"
28799,441,boiler-3,3,Boiler,19,BLR,boiler,None,NaN,None,1,None,None,None,None,[28803],[],"[{'id': 310108, 'building_id': 441, 'last_upda...","[boiler, hvac]"
28800,441,boiler-4,4,Boiler,19,BLR,boiler,None,NaN,None,1,None,None,None,None,[28803],[],"[{'id': 310144, 'building_id': 441, 'last_upda...","[boiler, hvac]"
28801,441,boiler-5,5,Boiler,19,BLR,boiler,None,NaN,None,1,None,None,None,None,[28803],[],"[{'id': 310152, 'building_id': 441, 'last_upda...","[boiler, hvac]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32354,399,chilledWaterPump,None,Pump,32,PUMP,pump,Chilled Water Pump,20.0,chilledWaterPump,1,None,None,None,None,[22751],[],"[{'id': 231126, 'building_id': 399, 'last_upda...","[pump, hvac, chilledWaterPump]"
32355,399,chilledWaterPump-4,4,Pump,32,PUMP,pump,Chilled Water Pump,20.0,chilledWaterPump,1,None,None,None,None,[22751],[],"[{'id': 359904, 'building_id': 399, 'last_upda...","[pump, hvac, chilledWaterPump]"
32356,399,condenser,None,Condenser,22,COND,condenser,None,NaN,None,1,None,None,None,None,[],[],"[{'id': 231136, 'building_id': 399, 'last_upda...","[hvac, condenser]"


In [11]:
all_equipment['equip_type_name'].value_counts()

Air Handling Unit            1144
Meter                         800
Fan                           776
Variable Air Volume           715
Pump                          509
Site Sensors                  372
Fan Coil Unit                 366
Unit Ventilator               193
Chiller                       155
Electrical Panel              153
Boiler                        138
Lighting System                76
Radiant System                 70
Cooling Tower                  65
Virtual                        65
Battery                        61
Hot Water Plant                60
Heat Exchanger                 57
Chilled Water Plant            51
Heat Pump                      34
Condenser                      18
Elevator                       16
Variable Refrigerant Flow      10
Duct Heater                     3
Tank                            2
Dry Cooler                      1
Name: equip_type_name, dtype: int64

In [12]:
all_measurements = pd.json_normalize(client.get_all_measurements())
all_measurements[['id', 'name']].set_index('id')

,name
id,
10,Occupancy
57,Dimensionless
11,On/Off
17,Motor Speed
31,Torque
7,Position
12,Enable
15,Open/Close
26,State


In [13]:
point_types = pd.json_normalize(client.get_all_point_types())
point_types.join(all_measurements[['id', 'name']]
                 .set_index('id'), on='measurement_id').set_index('id')[['tag_name', 'name']].rename(columns={
    'name': 'measurement_type',
    'tag_name': 'point_type'
})
point_types

,id,tag_name,active,measurement_id,tag_set_ids,tags,default_unit_id
0,124,Occupied Heating Setpoint,True,1,"[15, 2, 4, 62, 51, 55]","[air, sp, temp, zone, heating, occ]",None
1,118,Outside Air Carbon Dioxide,True,6,"[15, 8, 1, 24]","[air, co2, sensor, outside]",None
2,130,Return Air Temperature Setpoint,True,1,"[15, 2, 4, 63]","[air, sp, temp, return]",None
3,84,Dual-Temp Coil Discharge Air Temperature,True,1,"[15, 36, 47, 1, 4, 250]","[air, discharge, dualTemp, sensor, temp, coil]",None
4,81,Reheat Coil Discharge Air Temperature,True,1,"[15, 36, 44, 1, 4, 250]","[air, discharge, reheats, sensor, temp, coil]",None
...,...,...,...,...,...,...,...
647,754,Evaporator Water Return Temperature,True,1,"[16, 40, 1, 4, 98]","[water, entering, sensor, temp, evaporator]",None
648,755,Evaporator Water Supply Temperature,True,1,"[16, 39, 1, 4, 98]","[water, leaving, sensor, temp, evaporator]",None
649,756,Water Supply Pressure,True,3,"[16, 39, 1, 6]","[water, leaving, sensor, pressure]",None
650,757,Water Return Pressure,True,3,"[16, 40, 1, 6]","[water, entering, sensor, pressure]",None


In [14]:
if use_local:
    with open('../api/points/all.json', 'r') as f:
        j = json.load(f)
        all_points = pd.json_normalize(j).set_index('id')
else:
    all_points = pd.json_normalize(client.get_all_points()).set_index('id')
all_points

,building_id,last_updated,first_updated,device,network_device,objectId,name,description,units,tagged_units,...,measurement_id,datasource_hash,topic,state_text,equip_id,state_text.1,state_text.2,state_text.3,state_text.4,state_text.5
id,,,,,,,,,,,,,,,,,,,,,
310029,441,1.560571e+12,1.550542e+12,None,None,None,245 E 149th St. Boiler-1 Fault Code,None,?,?,...,18,b115b5053493748f7076ad09f69c4456,nyserda+0000088737+boiler-1+245 E 149th St. Bo...,NaN,28797,NaN,NaN,NaN,NaN,NaN
310009,441,1.560485e+12,1.550701e+12,None,None,None,225 E 149th St. Boiler-1 Exhaust Temp,None,F,f,...,1,4a1a11502acee3078352da312c40c262,nyserda+0000088737+boiler-1+225 E 149th St. Bo...,NaN,28797,NaN,NaN,NaN,NaN,NaN
310010,441,1.560571e+12,1.550504e+12,None,None,None,225 E 149th St. Boiler-1 Fault Code,None,?,?,...,18,55882633fe8fb5aac7dac643c99b1af5,nyserda+0000088737+boiler-1+225 E 149th St. Bo...,NaN,28797,NaN,NaN,NaN,NaN,NaN
310011,441,1.560485e+12,1.550504e+12,None,None,None,225 E 149th St. Boiler-1 FFWD Temp,None,?,?,...,18,831a8df09b848f53e807fa0204c45eaa,nyserda+0000088737+boiler-1+225 E 149th St. Bo...,NaN,28797,NaN,NaN,NaN,NaN,NaN
310012,441,1.560485e+12,1.550504e+12,None,None,None,225 E 149th St. Boiler-1 Fire Rate In,None,?,?,...,18,aa147c829c7ed55b6e93cdbbb8b43d55,nyserda+0000088737+boiler-1+225 E 149th St. Bo...,NaN,28797,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359906,399,1.596254e+12,1.580533e+12,None,None,None,Main Meter kW_v1,None,None,kw,...,13,66e321814268c37187fe46120a91ea01,nyserda+0000097151+elecMeter-site+Main Meter k...,NaN,32358,NaN,NaN,NaN,NaN,NaN
359905,399,1.589532e+12,1.580533e+12,None,None,None,Main Meter kW_v0,None,None,kw,...,13,0d67a5188df9184da502b013c6d9ae11,nyserda+0000097151+elecMeter-site+Main Meter k...,NaN,32358,NaN,NaN,NaN,NaN,NaN
231120,399,1.543640e+12,1.528909e+12,,None,,Building Demand_value,Building Demand_value,kW,?,...,18,26ace99b6d6d2692b3d5ae723f489241,nyserda+0000097151+Site-Sensor+Building Demand...,NaN,32358,NaN,NaN,NaN,NaN,NaN


## What kinds of equipment are common across all buildings?
- How many buildings have which particular pieces of equipment?
- What buildings have the most data?
- What building is the most representative?
- What data has the most buildings?

In [15]:
# Buildings with the most points
all_points['building_id'].value_counts().head(20)

420    6068
354    2507
119    1980
259    1681
101    1405
441    1293
162    1224
286    1183
470    1140
438    1111
324    1108
426     959
452     867
190     777
387     713
417     599
418     506
423     463
378     444
279     415
Name: building_id, dtype: int64

_Building 420 has the most data points, nice._

In [16]:
# Top 20 Buildings with the most equipment
all_equipment['building_id'].value_counts().head(20)

420    342
418    222
438    221
259    218
470    205
354    197
452    189
162    179
423    139
134    129
99     126
118    122
120    120
119    111
190    110
487     94
426     92
230     91
489     85
384     84
Name: building_id, dtype: int64

In [17]:
equipment_names = all_equipment.set_index('id')[['equip_type_name', 'equip_type_tag', 'building_id',
                                                 'equip_subtype_name']]

all_points.join(equipment_names, on='equip_id', rsuffix='equipment')['equip_type_name'].value_counts()

Air Handling Unit            11342
Variable Air Volume           9554
Site Sensors                  4450
Fan Coil Unit                 3934
Virtual                       2186
Unit Ventilator               1989
Fan                           1935
Chiller                       1574
Pump                          1524
Meter                         1267
Boiler                        1227
Hot Water Plant                669
Chilled Water Plant            488
Heat Pump                      435
Heat Exchanger                 403
Radiant System                 386
Cooling Tower                  300
Electrical Panel               273
Lighting System                106
Condenser                       78
Battery                         61
Elevator                        29
Variable Refrigerant Flow       24
Dry Cooler                      15
Tank                             8
Duct Heater                      5
Name: equip_type_name, dtype: int64

In [18]:
points_w_meas_names = all_points.join(equipment_names, on='equip_id', rsuffix='equipment')
meter_equipment = points_w_meas_names.loc[points_w_meas_names['equip_type_name'] == 'Meter'][['building_id',
                                                         'equip_type_name',
                                                         'name',
                                                         'description',
                                                         'units']]
meter_equipment

,building_id,equip_type_name,name,description,units
id,,,,,
344965,441,Meter,225 E 149th St. Water Meter Water Meter,None,gal
344964,441,Meter,225 E 149th St. Basement Common Meter Water Ga...,None,gal
344963,441,Meter,245 E 149th St. Water Meter Water Meter,None,gal
344962,441,Meter,245 E 149th St. Water Meter Gallons Total,None,gal
203559,191,Meter,,Main Power,kWh
...,...,...,...,...,...
359906,399,Meter,Main Meter kW_v1,None,None
359905,399,Meter,Main Meter kW_v0,None,None
231120,399,Meter,Building Demand_value,Building Demand_value,kW


In [19]:
site_sensor_equipment = points_w_meas_names.loc[points_w_meas_names['equip_type_name'] == 'Site Sensors'][['building_id',
                                                                'equip_type_name',
                                                                'name',
                                                                'description',
                                                                'units']]
site_sensor_equipment

,building_id,equip_type_name,name,description,units
id,,,,,
310850,441,Site Sensors,235 E 149th St. Floor-07 Apartment-J APT 07J T...,None,F
310287,441,Site Sensors,225 E 149th St. Floor-03 Apartment-M APT 03M T...,None,F
310350,441,Site Sensors,225 E 149th St. Floor-05 Apartment-J APT 05J T...,None,F
310397,441,Site Sensors,225 E 149th St. Floor-06 Apt-06E Zone Zone Tem...,None,F
310426,441,Site Sensors,225 E 149th St. Floor-07 Apt-07A Zone Zone Tem...,None,F
...,...,...,...,...,...
230932,398,Site Sensors,,Outdoor Conditions (CRAC 17-1)_OaRh,%
230936,398,Site Sensors,,Outdoor Conditions (CRAC 5-1)_OaTmp,F
230943,398,Site Sensors,,Outdoor Conditions (CRAC-23-1)_OaWetBulbTmp,F
